In [1]:
'''
=====================================================================
# "프로젝트 : 모든 장르 간 편향성 측정해 보기"
=====================================================================

[프로젝트 목표]
1) 전 장르 × 영화 구분(예술/일반/독립)의 내재 편향을 한 번에 보기
    - 모든 타깃쌍(X vs Y)에 대해 장르별 WEAT 효과크기(=Cohen's d)를 계산하고 히트맵으로 비교

2) 대표 단어 셋 개선으로 해석력 높이기
    - TF-IDF의 중복/길이 편향 문제를 줄이기 위해
      [로그-오즈(Dirichlet 사전) → 상위 후보] + [MMR 중복 억제]를 적용

3) 어디서나 돌아가는 실행성/재현성
    - konlpy(Mecab > Okt), 한글 폰트 자동탐색(실패시 영어 라벨 폴백)
    - 난수 시드 고정, 결과 CSV/PNG 저장

4) 독립영화 코퍼스가 없어도 분석 지속
    - 실제 파일이 없으면 장르 전체 토큰 − (예술/일반)으로 코퍼스 합성

[핵심 설계]
    - 명사 중심 토큰화(Mecab>Okt 폴백)
    - 대표어 : 로그-오즈(Dirichlet 사전)로 ‘해당 클래스에 특이한 단어’ 선별 > MMR로 중복 억제
    - 타깃쌍(예술, 일반) × 장르별 WEAT 계산 > 히트맵 저장
    - 폰트 이슈 자동 처리(한글 폰트 탐색, 실패 시 영문으로)
    
[참고]
- WEAT: Caliskan et al. (2017)

=====================================================================
# "프로젝트 회고"
=====================================================================

[개인적인 해석]
    - 대표어의 질이 전부다. 대표어가 장르의 개념축을 잘 잡아야 WEAT 신호가 또렷하다.
    - 상관 = 인과(x), WEAT는 분포 상의 연관성이다. 제작,마케팅, 데이터 수집 편향이 모두 섞일 수 있다.
    - WEAT 부호
        > WEAT(예술 vs 일반)이 양수이면: 예술 타깃 단어들이 해당 장르 대표어(A)에 더 가깝다는 신호.
        > 음수이면 반대(일반이 더 가깝다).

[한계와 리스크]
- 독립영화 합성 편향 : 잔여 토큰으로 만든 코퍼스는 실제 독립영화의 언어적 특질을 완벽히 반영하지 않을 수 있음.
    > 가능하면 실제 독립영화 텍스트를 확보하고 만들어야 함
- 명사 한정 : 형용사/동사(감성·스타일 지표)가 빠져 미세한 스타일 차가 덜 잡힐 수 있음.
- MMR_SELECT_K, ALPHA_DIR, min_count, window 등에 따라 결과가 변동

[회고]
데이터의 대표어를 잡아는 것을 처음해봤다. 처음에는 TF-IDF로는 비슷한 단어가 우르르 몰려와서 다른 알고리즘을 찾아봤는데,
로그-오즈, MMR을 돌리니 장르마다 다양한 축이 나타났다. 생각의 폭이 넓어지는 느낌.
다만 독립영화는 결국 타협점을 찾아서 했고, 실제 데이터가 있을 떄 다시 해봐야 할 것 같다.
마지막으로, 시각화하는데 자꾸 ㅁㅁㅁ 이렇게 떠서 스트레스를 받았다. 그래서 폰트 설정하는 법을 배웠다.


=====================================================================
'''

# ==========================
# 필요한 라이브러리 Import
# ==========================
from __future__ import annotations
import os, re, math, json, random, unicodedata, warnings
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Iterable, Set
from itertools import combinations

import numpy as np
import pandas as pd

# 임베딩
from gensim.models import Word2Vec

# 형태소 분석기 폴백(Mecab > Okt)
# 환경 의존도 낮추기: Mecab이 없을 때도 파이프라인이 멈추지 않도록 안전 폴백
try:
    from konlpy.tag import Mecab
    _HAS_MECAB = True
except Exception:
    _HAS_MECAB = False
    from konlpy.tag import Okt

import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

RANDOM_SEED = 42               # 재현성 결과 재현을 위해 모든 난수 고정
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
# ==========================
# 경로/파일 불러오기
# ==========================
DATA_DIR = os.path.join(os.getenv("HOME"), "work/weat/data")

# 데이터 파일
genre_txt = [
    'synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt',
    'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt',
    'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt',
    'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt',
    'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt'
]

genre_name = [
    'SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극',
    '서부극(웨스턴)', '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지'
]

# 영화 구분 타깃(독립은 있으면)
# 최소 타깃(예술/일반)은 필수. 독립은 있으면 사용
TARGET_TXT = {
    '예술영화': 'synopsis_art.txt',
    '일반영화': 'synopsis_gen.txt',
    # '독립영화': 'synopsis_indie.txt',  # 존재하면 인식
}
OPTIONAL_INDEPENDENT_FILENAME = 'synopsis_indie.txt'  # 있으면 사용, 없으면 합성

# 대표단어 셋 하이퍼파라미터
TOPN_CANDIDATES = 200 
MMR_SELECT_K   = 40     # 최종 대표단어 개수 (25~50)
MIN_TERM_LEN   = 2
MIN_DF         = 5 
ALPHA_DIR      = 0.01
MMR_LAMBDA     = 0.65   # MMR 관련성/다양성 균형(0.5~0.8)

# Word2Vec 설정
W2V_VECTOR_SIZE = 300
W2V_WINDOW      = 5
W2V_MIN_COUNT   = 5
W2V_WORKERS     = 4
W2V_SG          = 1   # 1: skip-gram(희소 단어에 강함), 0: CBOW
W2V_EPOCHS      = 10

WEAT_PERMUTATIONS = 0  # 예: 1000

# 출력 파일
OUT_CSV = 'weat_genre_bias_all_pairs.csv'
OUT_FIG = 'weat_genre_heatmap_all_pairs.png'

# 플롯 라벨 언어 정책
# 한글 폰트 부재 환경에서도 결과 해석 가능하도록 자동 폴백
PLOT_LABEL_LANG = 'auto'  # 'auto', 'ko', 'en'

# 한글 > 영어 매핑(축 라벨)
GENRE_EN = {
    'SF':'Sci-Fi','가족':'Family','공연':'Performance','공포(호러)':'Horror','기타':'Misc',
    '다큐멘터리':'Documentary','드라마':'Drama','멜로로맨스':'Romance','뮤지컬':'Musical',
    '미스터리':'Mystery','범죄':'Crime','사극':'Historical','서부극(웨스턴)':'Western',
    '성인물(에로)':'Adult','스릴러':'Thriller','애니메이션':'Animation','액션':'Action',
    '어드벤처':'Adventure','전쟁':'War','코미디':'Comedy','판타지':'Fantasy'
}
TARGET_EN = {'예술영화':'Art','일반영화':'General','독립영화':'Indie'}

# ==========================
# 토큰화/정규화
# ==========================
_ALNUM_RE  = re.compile(r'^[가-힣A-Za-z0-9]+$')

def normalize_token(tok: str) -> str:
    tok = tok.strip()
    tok = unicodedata.normalize('NFC', tok)
    tok = re.sub(r'[^가-힣A-Za-z0-9]', '', tok)  # 기호 제거
    return tok

def is_good_token(tok: str) -> bool:
    if len(tok) < MIN_TERM_LEN:
        return False
    if not _ALNUM_RE.match(tok):
        return False
    if tok.isdigit():
        return False
    return True

def get_tokenizer():
    """
    [의도] 품사 기반 '명사' 중심 추출로 장르/구분의 핵심 개념축을 잡는다.
    - Mecab이 있으면 정확한 품사 태깅 사용
    - 없으면 Okt.nouns()로 폴백(실행성 우선)
    """
    try:
        from konlpy.tag import Mecab
        mecab = Mecab()
        def _tok(text: str) -> List[str]:
            return [w for w, t in mecab.pos(text) if t.startswith('NN')]
        print('[Tokenizer] Using Mecab (NN* only)')
        return _tok
    except Exception as e:
        print(f'[Tokenizer] Mecab unavailable → fallback to Okt.nouns(): {e}')
        from konlpy.tag import Okt
        okt = Okt()
        def _tok(text: str) -> List[str]:
            return okt.nouns(text)
        print('[Tokenizer] Using Okt.nouns() (fallback)')
        return _tok

def read_lines(path: str) -> List[str]:
    """입력 경로 체크, 공백 라인 제거"""
    if not os.path.exists(path):
        warnings.warn(f'파일 없음: {path}')
        return []
    with open(path, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    return lines

def tokenize_file(path: str, tokenizer) -> List[List[str]]:
    """문장 단위로 명사 토큰화, 정규화, 노이즈 제거"""
    sents = read_lines(path)
    out: List[List[str]] = []
    for s in sents:
        toks = [normalize_token(t) for t in tokenizer(s)]
        toks = [t for t in toks if is_good_token(t)]
        if toks:
            out.append(toks)
    return out

# ==========================
# 폰트 처리 (시각화 자동 폴백)
# ==========================
def try_enable_korean_font() -> bool:
    rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지
    candidates = [
        'AppleGothic',                          # macOS
        'Malgun Gothic', '맑은 고딕',           # Windows
        'NanumGothic', 'NanumGothicCoding', 'NanumBarunGothic',  # Linux
        'Noto Sans CJK KR', 'Noto Sans KR',
        'UnDotum', '돋움'
    ]
    for name in candidates:
        try:
            _ = font_manager.findfont(font_manager.FontProperties(family=name), fallback_to_default=False)
            rcParams['font.family'] = name
            print(f'[Font] Using Korean font: {name}')
            return True
        except Exception:
            continue
    print('[Font] No Korean font found. Will fall back to English labels.')
    return False

# ==========================
# 임베딩 학습
# ==========================
def train_word2vec(all_sentences: List[List[str]]) -> Word2Vec:
    """
         장르, 타깃 전체 문장을 한 번에 학습해 '공용 임베딩 공간'을 만든다.
         > X/Y/A/B 단어 간 상대 거리 비교가 일관되고 안정적
    """
    print(f'[W2V] Training on {len(all_sentences):,} sentences...')
    model = Word2Vec(
        sentences=all_sentences,
        vector_size=W2V_VECTOR_SIZE,
        window=W2V_WINDOW,
        min_count=W2V_MIN_COUNT,
        workers=W2V_WORKERS,
        sg=W2V_SG,
        seed=RANDOM_SEED,
        epochs=W2V_EPOCHS,
    )
    return model

# ==========================
# 대표어 스코어링: 로그-오즈 + MMR
# ==========================
def counts_from_docs(docs: List[List[str]]) -> Counter:
    """클래스 내 단어 수(BoW)를 집계 > 로그오즈 계산의 입력"""
    c = Counter()
    for sent in docs:
        c.update(sent)
    return c

def log_odds_dirichlet(class_counts: Counter, other_counts: Counter, alpha: float = ALPHA_DIR) -> Dict[str, float]:
    """
         TF-IDF의 한계(TF/길이 편향, 동의어 중복)를 보완하기 위해
         '그 클래스에 얼마나 특이한가'를 z-score 근사로 계산
    """
    vocab: Set[str] = set(class_counts) | set(other_counts)
    V = len(vocab)
    n_c = sum(class_counts.values())
    n_o = sum(other_counts.values())

    zscores: Dict[str, float] = {}
    for w in vocab:
        f_c = class_counts.get(w, 0)
        f_o = other_counts.get(w, 0)
        # 로짓 차이(사전 alpha로 스무딩)
        num_c = f_c + alpha
        den_c = (n_c - f_c) + alpha * (V - 1)
        num_o = f_o + alpha
        den_o = (n_o - f_o) + alpha * (V - 1)
        delta = math.log(num_c / den_c) - math.log(num_o / den_o)
        var = 1.0 / num_c + 1.0 / num_o
        z = delta / math.sqrt(var)
        zscores[w] = z
    return zscores

def mmr_select(candidates: List[str], scores: Dict[str, float], model: Word2Vec,
               k: int = MMR_SELECT_K, lamb: float = MMR_LAMBDA) -> List[str]:
    """
         MMR(Maximal Marginal Relevance)로 유사한 단어의 중복 선정을 억제
         '다양성 있는' 대표어 셋을 구성해 WEAT의 해석력을 높임
    """
    cand = [w for w in candidates if w in model.wv]
    if not cand:
        return []
    vals = np.array([scores[w] for w in cand], dtype=float)
    if np.ptp(vals) == 0:
        rel = {w: 0.5 for w in cand}
    else:
        vmin, vmax = float(vals.min()), float(vals.max())
        rel = {w: (scores[w] - vmin) / (vmax - vmin) for w in cand}

    selected: List[str] = []
    while len(selected) < min(k, len(cand)):
        best_w, best_score = None, -1e9
        for w in cand:
            if w in selected:
                continue
            if not selected:
                redundancy = 0.0
            else:
                v = model.wv[w]
                sims = []
                for s in selected:
                    svec = model.wv[s]
                    denom = (np.linalg.norm(v) * np.linalg.norm(svec)) or 1e-9
                    sims.append(float(np.dot(v, svec) / denom))
                redundancy = max(sims) if sims else 0.0
            mmr = lamb * rel[w] - (1.0 - lamb) * redundancy
            if mmr > best_score:
                best_score, best_w = mmr, w
        assert best_w is not None
        selected.append(best_w)
    return selected

def build_representatives(class_docs: Dict[str, List[List[str]]], model: Word2Vec,
                          topn_candidates: int = TOPN_CANDIDATES, mmr_k: int = MMR_SELECT_K,
                          min_df: int = MIN_DF, alpha: float = ALPHA_DIR) -> Dict[str, List[str]]:
    """
         각 클래스 마다
         BoW 집계 > 로그오즈 점수 > 상위 후보 추출 > MMR로 중복 억제
    """
    # 전체 카운트(다른 집단 비교용)
    global_counts = counts_from_docs([tok for docs in class_docs.values() for tok in docs])
    reps: Dict[str, List[str]] = {}
    for cname, docs in class_docs.items():
        c_counts = counts_from_docs(docs)
        o_counts = global_counts.copy()
        # other = global - class (음수 방지 보정)
        for w, f in c_counts.items():
            if w in o_counts:
                o_counts[w] = max(0, o_counts[w] - f)
        # 희소 토큰 컷
        c_counts = Counter({w: f for w, f in c_counts.items() if f >= min_df})
        # 점수
        z = log_odds_dirichlet(c_counts, o_counts, alpha=alpha)
        # 상위 후보
        cand = [w for w, _ in sorted(z.items(), key=lambda x: x[1], reverse=True)]
        cand = [w for w in cand if w in c_counts][:topn_candidates]
        # MMR로 최종 대표어
        reps[cname] = mmr_select(cand, z, model, k=mmr_k)
    return reps

# ==========================
# WEAT 계산
# ==========================
def cos_sim(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) or 1e-9
    return float(np.dot(a, b) / denom)

def mean_cos_sim(word: str, S: Iterable[str], model: Word2Vec) -> float:
    if word not in model.wv:
        return 0.0
    v = model.wv[word]
    sims = []
    for s in S:
        if s in model.wv:
            sims.append(cos_sim(v, model.wv[s]))
    return float(np.mean(sims)) if sims else 0.0

def s_wAB(word: str, A: Iterable[str], B: Iterable[str], model: Word2Vec) -> float:
    return mean_cos_sim(word, A, model) - mean_cos_sim(word, B, model)

def weat_effect_size(X: Iterable[str], Y: Iterable[str], A: Iterable[str], B: Iterable[str], model: Word2Vec) -> float:
    X, Y, A, B = list(X), list(Y), list(A), list(B)
    X = [w for w in X if w in model.wv]
    Y = [w for w in Y if w in model.wv]
    A = [w for w in A if w in model.wv]
    B = [w for w in B if w in model.wv]
    if not X or not Y or not A or not B:
        return float('nan')
    sX = np.array([s_wAB(w, A, B, model) for w in X])
    sY = np.array([s_wAB(w, A, B, model) for w in Y])
    num = np.mean(sX) - np.mean(sY)
    denom = np.std(np.concatenate([sX, sY], axis=0), ddof=1) or 1e-9
    return float(num / denom)

def weat_pvalue_permutation(X: List[str], Y: List[str], A: List[str], B: List[str],
                            model: Word2Vec, n_perm: int = 1000) -> float:
    if n_perm <= 0:
        return np.nan
    X = [w for w in X if w in model.wv]
    Y = [w for w in Y if w in model.wv]
    A = [w for w in A if w in model.wv]
    B = [w for w in B if w in model.wv]
    if not X or not Y or not A or not B:
        return np.nan
    all_words = X + Y
    observed = weat_effect_size(X, Y, A, B, model)
    count = 0
    for _ in range(n_perm):
        random.shuffle(all_words)
        Xp = all_words[:len(X)]
        Yp = all_words[len(X):]
        val = weat_effect_size(Xp, Yp, A, B, model)
        if abs(val) >= abs(observed):
            count += 1
    return (count + 1) / (n_perm + 1)

# ==========================
# 독립영화 자동 합성
# ==========================
def bow_to_sentences(counter: Counter, sentence_len: int = 10, max_sentences: int | None = None) -> List[List[str]]:
    tokens = []
    for w, f in counter.items():
        tokens.extend([w] * int(f))
    if not tokens:
        return []
    random.shuffle(tokens)
    sents: List[List[str]] = []
    for i in range(0, len(tokens), sentence_len):
        chunk = tokens[i:i+sentence_len]
        if len(chunk) >= max(3, sentence_len//2):  # 너무 짧은 문장 방지
            sents.append(chunk)
        if max_sentences is not None and len(sents) >= max_sentences:
            break
    return sents

def synthesize_indie_docs(genre_docs: Dict[str, List[List[str]]],
                          target_docs_partial: Dict[str, List[List[str]]]) -> List[List[str]]:
    """
        '독립영화' 코퍼스가 없을 시에도 분석을 진행할 수 있게,
         장르 전체 토큰, 예술+일반 토큰 = 잔여 토큰으로 코퍼스 생성
    """
    # 장르 전체 BoW
    global_counts = counts_from_docs([s for docs in genre_docs.values() for s in docs])
    known_counts = Counter()
    for k in ['예술영화', '일반영화']:
        if k in target_docs_partial:
            known_counts.update(counts_from_docs(target_docs_partial[k]))
    indie_counts = Counter()
    for w, f in global_counts.items():
        indie_counts[w] = max(0, f - known_counts.get(w, 0))
    indie_counts = Counter({w: f for w, f in indie_counts.items() if f >= MIN_DF})
    # 문장 합성
    indie_docs = bow_to_sentences(indie_counts, sentence_len=10, max_sentences=50000)
    return indie_docs

# ==========================
# 메인 파이프라인
# ==========================
def main():
    tokenizer = get_tokenizer()

    # 1) 장르 코퍼스 로딩 & 토큰화
    genre_files = {g: os.path.join(DATA_DIR, f) for g, f in zip(genre_name, genre_txt)}
    genre_docs: Dict[str, List[List[str]]] = {}

    for g, fpath in genre_files.items():
        docs = tokenize_file(fpath, tokenizer)
        if len(docs) == 0:
            warnings.warn(f'장르 "{g}"에서 유효 문서가 없습니다: {fpath}')
        genre_docs[g] = docs
        print(f'[Genre] {g:10s}: {len(docs):6d} sentences')

    # 2) 타깃(영화 구분) 코퍼스 로딩 & 토큰화
    target_docs: Dict[str, List[List[str]]] = {}
    for k, fname in TARGET_TXT.items():
        fpath = os.path.join(DATA_DIR, fname)
        docs = tokenize_file(fpath, tokenizer)
        if len(docs) == 0:
            warnings.warn(f'타깃 "{k}"에서 유효 문서가 없습니다: {fpath}')
        target_docs[k] = docs
        print(f'[Target] {k:10s}: {len(docs):6d} sentences')

    # 독립영화: 파일 있으면 사용, 없으면 합성
    indie_path = os.path.join(DATA_DIR, OPTIONAL_INDEPENDENT_FILENAME)
    if os.path.exists(indie_path):
        indie_docs = tokenize_file(indie_path, tokenizer)
        target_docs['독립영화'] = indie_docs
        print(f'[Target] {"독립영화":10s}: {len(indie_docs):6d} sentences (from file)')
    else:
        indie_docs = synthesize_indie_docs(genre_docs, target_docs_partial=target_docs)
        if indie_docs:
            target_docs['독립영화'] = indie_docs
            print(f'[Target] {"독립영화":10s}: {len(indie_docs):6d} sentences (synthesized)')

    # 최소 두 타깃 이상 필요
    if len(target_docs) < 2:
        raise RuntimeError('타깃(영화 구분) 코퍼스가 2개 미만입니다. synopsis_art.txt / synopsis_gen.txt를 확인하세요.')

    # 3) 임베딩 학습
    all_sentences = [s for docs in genre_docs.values() for s in docs] + [s for docs in target_docs.values() for s in docs]
    if not all_sentences:
        raise RuntimeError('학습할 문장이 없습니다. 파일 경로/인코딩을 확인하세요.')
    w2v = train_word2vec(all_sentences)

    # 4) 대표어 선정
    genre_reps  = build_representatives(genre_docs,  w2v, topn_candidates=TOPN_CANDIDATES, mmr_k=MMR_SELECT_K)
    target_reps = build_representatives(target_docs, w2v, topn_candidates=TOPN_CANDIDATES, mmr_k=MMR_SELECT_K)

    # 5) 모든 타깃쌍에 대해, 각 장르 g의 WEAT 계산
    rows = []
    tgt_keys = list(target_reps.keys())
    tgt_pairs = list(combinations(tgt_keys, 2))
    # 기타 장르 풀(장르 g 이외의 대표어 모음) → A와 크기 맞춘 B를 MMR로 구성
    all_other_pool = {g: [w for gg, ws in genre_reps.items() if gg != g for w in ws] for g in genre_reps}

    for g in genre_name:
        row = {'genre': g, 'A_words': ','.join(genre_reps.get(g, [])[:10])}
        A = genre_reps.get(g, [])
        for (t1, t2) in tgt_pairs:
            col_name = f'WEAT({t1} vs {t2})'
            if not A:
                row[col_name] = np.nan
                pcol = 'p_value' if WEAT_PERMUTATIONS > 0 else 'p_value(NA)'
                row[f'{pcol}({t1} vs {t2})'] = np.nan
                continue
            pool = [w for w in all_other_pool.get(g, []) if w not in set(A)]
            pseudo_scores = {w: 1.0 for w in pool}
            B = mmr_select(pool, pseudo_scores, w2v, k=len(A)) if pool else []
            X = target_reps.get(t1, [])
            Y = target_reps.get(t2, [])
            score = weat_effect_size(X, Y, A, B, w2v)
            pval = weat_pvalue_permutation(X, Y, A, B, w2v, n_perm=WEAT_PERMUTATIONS)
            row[col_name] = score
            pcol = 'p_value' if WEAT_PERMUTATIONS > 0 else 'p_value(NA)'
            row[f'{pcol}({t1} vs {t2})'] = pval
        rows.append(row)

    df = pd.DataFrame(rows).set_index('genre')
    df.to_csv(OUT_CSV, encoding='utf-8-sig')
    print(f'[OUT] Saved: {OUT_CSV}')

    # 6) 히트맵 시각화
    weat_cols = [c for c in df.columns if c.startswith('WEAT(')]
    vals = df[weat_cols].values.astype(float)

    maxabs = np.nanmax(np.abs(vals)) if np.isfinite(vals).any() else 1.0
    if not np.isfinite(maxabs) or maxabs == 0:
        maxabs = 1.0

    # 라벨 언어 결정 (한글 폰트가 없으면 영어로)
    if PLOT_LABEL_LANG == 'ko':
        use_korean = True
        _ = try_enable_korean_font()
    elif PLOT_LABEL_LANG == 'en':
        use_korean = False
    else:  # 모두 아닐 경우 auto
        use_korean = try_enable_korean_font()

    # y축(장르)
    ylabels = df.index.tolist() if use_korean else [GENRE_EN.get(g, g) for g in df.index.tolist()]

    # x축(WEAT쌍)
    xlabels = []
    for c in weat_cols:
        m = re.match(r'WEAT\((.+) vs (.+)\)', c)
        if m:
            a, b = m.group(1), m.group(2)
            if not use_korean:
                a = TARGET_EN.get(a, a)
                b = TARGET_EN.get(b, b)
            xlabels.append(f'WEAT({a} vs {b})')
        else:
            xlabels.append(c)

    fig_w = max(6, 2 + 1.5 * len(weat_cols))
    fig_h = max(6, len(df) * 0.35)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    im = ax.imshow(vals, aspect='auto', cmap='coolwarm', vmin=-maxabs, vmax=+maxabs)

    # 축 라벨
    ax.set_yticks(range(len(df)))
    ax.set_yticklabels(ylabels, fontsize=9)
    ax.set_xticks(range(len(weat_cols)))
    ax.set_xticklabels(xlabels, rotation=0, fontsize=10)

    for i in range(vals.shape[0]):
        for j in range(vals.shape[1]):
            v = vals[i, j]
            txt = 'nan' if np.isnan(v) else f'{v:.2f}'
            ax.text(j, i, txt, ha='center', va='center', fontsize=8)

    cbar = fig.colorbar(im, ax=ax, fraction=0.025, pad=0.02)
    cbar.set_label('WEAT effect size (Cohen d, centered at 0)', rotation=90)

    title_ko = '장르별 WEAT 효과크기 (타깃: 영화 구분 — 모든 쌍)'
    title_en = 'WEAT effect sizes by genre (targets: film categories — all pairs)'
    ax.set_title(title_ko if use_korean else title_en, fontsize=12)

    fig.tight_layout()
    fig.savefig(OUT_FIG, dpi=220)
    print(f'[OUT] Saved: {OUT_FIG}')

    # 7) 대표어 결과 미리보기(빠른 검시)
    print('\n[장르 대표어 미리보기]')
    for g in genre_name:
        print(f'- {g:10s}:', ', '.join(genre_reps.get(g, [])[:15]))
    print('\n[타깃 대표어 미리보기]')
    for k in tgt_keys:
        print(f'- {k}: {", ".join(target_reps.get(k, [])[:20])}')

if __name__ == '__main__':
    main()


[Tokenizer] Mecab unavailable → fallback to Okt.nouns(): Install MeCab in order to use it: http://konlpy.org/en/latest/install/
[Tokenizer] Using Okt.nouns() (fallback)
[Genre] SF        :   1020 sentences
[Genre] 가족        :    416 sentences
[Genre] 공연        :    334 sentences
[Genre] 공포(호러)    :   3063 sentences
[Genre] 기타        :   3625 sentences
[Genre] 다큐멘터리     :   7080 sentences
[Genre] 드라마       :  18988 sentences
[Genre] 멜로로맨스     :   5732 sentences
[Genre] 뮤지컬       :    168 sentences
[Genre] 미스터리      :   1066 sentences
[Genre] 범죄        :   1680 sentences
[Genre] 사극        :    238 sentences
[Genre] 서부극(웨스턴)  :     48 sentences
[Genre] 성인물(에로)   :   1625 sentences
[Genre] 스릴러       :   2246 sentences
[Genre] 애니메이션     :   8681 sentences
[Genre] 액션        :   5829 sentences
[Genre] 어드벤처      :    534 sentences
[Genre] 전쟁        :    386 sentences
[Genre] 코미디       :   4610 sentences
[Genre] 판타지       :    791 sentences
[Target] 예술영화      :  14286 sentences
[Target] 일반영화   